# 해야 할 일

- 논문 검색 (google scholar, ieee xplore, dbpia 등)<br/>
  (관련 키워드 : deep learning, detection, danger ....)
- 초록 작성 : 찾아본 논문들 참고해서 한글 혹은 영어로 작성
- 이미지 수집 및 전처리 (32,32,3 이미지 리사이징, 폴더별로 정리)

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(24, 24), # 우리는 32,32 로 할거니까 변경해줘야겠지 !
        batch_size=3,
        class_mode='binary') #우리는 위험하다 위험하지 않다니까 binary로 변경 
#         class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(24, 24),     # 우리는 32,32 로 할거니까 변경해줘야겠지 !
        batch_size=3,
        class_mode='binary') #우리는 위험하다 위험하지 않다니까 binary로 변경
#         class_mode='categorical')

# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3))) # 우리는 32,32 로 할거니까 변경해줘야겠지 !
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 
# 출력 결과는 0혹은1로 해야하니 1으로 변경, softmax는 0부터 1사이의 값으로 바꿔주는거니까 성능이 오히려 떨어질 수 있어서 sigmoid 로 변경
# 참고 URL :  https://m.blog.naver.com/PostView.nhn?blogId=infoefficien&logNo=221170205067&proxyReferer=https%3A%2F%2Fwww.google.com%2F

# 3. 모델 학습과정 설정하기
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=10, # 데이터가 30개고, 배치가 3이니까 한 스텝에서 10번 반복한다. 그럼 모든 데이터를 한번씩 다 학습하니까
        epochs=50,  # 전체 학습을 총 50번 한다.
        validation_data=test_generator,
        validation_steps=5)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Found 30 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/50
10/10 [==============================] - 1s 140ms/step - loss: 0.6455 - accuracy: 0.7667 - val_loss: 0.4044 - val_accuracy: 0.9231
Epoch 2/50
10/10 [==============================] - 0s 31ms/step - loss: 0.0985 - accuracy: 1.0000 - val_loss: 0.1204 - val_accuracy: 1.0000
Epoch 3/50
10/10 [==============================] - 0s 30ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0264 - val_accuracy: 1.0000
Epoch 4/50
10/10 [==============================] - 0s 30ms/step - loss: 2.5197e-04 - accuracy: 1.0000 - val_loss: 0.0184 - val_accuracy: 1.0000
Epoch 5/50
10/10 [==============================] - 0s 30ms/step - loss: 2.8591e-05 - accuracy: 1.0000 - val_loss: 0.0131 - val_accuracy: 1.0000
Epoch 6/50
10/10 [==============================] - 0s 31ms/step - loss: 1.4341e-05 - accuracy: 1.0000 - val_loss: 0.0114 - val_accuracy: 1.0000
Epoch 7/50
10/10 [==============================] - 0s 30ms/s